In [1]:
from IPython.core.display import clear_output

In [2]:
! apt-get update
! pip install -q pandas matplotlib 
clear_output()

In [3]:
!ls

__notebook_source__.ipynb


In [10]:
! wget https://huggingface.co/datasets/declare-lab/MELD/resolve/main/MELD.Raw.tar.gz
! tar -xf MELD.Raw.tar.gz && rm MELD.Raw.tar.gz

--2023-04-16 10:11:48--  https://huggingface.co/datasets/declare-lab/MELD/resolve/main/MELD.Raw.tar.gz
Resolving huggingface.co (huggingface.co)... 13.249.85.127, 13.249.85.16, 13.249.85.92, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.127|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e5/f9/e5f9b3280b3cb63549fb6376cd84286e8bf7cade60967f984e192dda5701e74b/a56b4407d574195cbce470d86f9c9d72fcfea59b0e34502ecd4babee4a5c613e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27MELD.Raw.tar.gz%3B+filename%3D%22MELD.Raw.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1681899109&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2U1L2Y5L2U1ZjliMzI4MGIzY2I2MzU0OWZiNjM3NmNkODQyODZlOGJmN2NhZGU2MDk2N2Y5ODRlMTkyZGRhNTcwMWU3NGIvYTU2YjQ0MDdkNTc0MTk1Y2JjZTQ3MGQ4NmY5YzlkNzJmY2ZlYTU5YjBlMzQ1MDJlY2Q0YmFiZWU0YTVjNjEzZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uP

In [11]:
! rm MELD.Raw.tar.gz

In [12]:
! pwd
! ls MELD.Raw/

/kaggle/working
README.txt  dev_sent_emo.csv  test_sent_emo.csv
dev.tar.gz  test.tar.gz       train.tar.gz


In [13]:
import os
os.chdir('./MELD.Raw/')
! tar -xf dev.tar.gz 
! rm dev.tar.gz 
! tar -xf test.tar.gz
! rm test.tar.gz
! tar -xf train.tar.gz
! rm train.tar.gz
os.chdir('..')


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [14]:
! ls ./MELD.Raw/train_splits | wc -l

9989


# Fetche Texts

In [16]:
from multiprocessing import Pool

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import glob
from pathlib import Path
import os

from tqdm.auto import tqdm

In [17]:
MAX_UTTERANCE_LENGTH = 20
MAX_NUM_WORDS = 20000

In [18]:
from sklearn.model_selection import train_test_split

train_utterances = pd.read_csv('./MELD.Raw/train_sent_emo.csv')
test_utterances  = pd.read_csv('./MELD.Raw/test_sent_emo.csv')
dev_utterances =  pd.read_csv('./MELD.Raw/dev_sent_emo.csv')
# train_utterances.head()
test_utterances.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"


In [19]:
print(len(dev_utterances))
! ls MELD.Raw/dev_splits_complete | wc -l
! ls MELD.Raw/dev_splits_complete 

1109
1112
dia0_utt0.mp4	  dia23_utt0.mp4   dia50_utt0.mp4   dia76_utt13.mp4
dia0_utt1.mp4	  dia23_utt1.mp4   dia50_utt1.mp4   dia76_utt14.mp4
dia100_utt0.mp4   dia23_utt10.mp4  dia50_utt2.mp4   dia76_utt15.mp4
dia101_utt0.mp4   dia23_utt11.mp4  dia50_utt3.mp4   dia76_utt16.mp4
dia102_utt0.mp4   dia23_utt12.mp4  dia51_utt0.mp4   dia76_utt2.mp4
dia102_utt1.mp4   dia23_utt13.mp4  dia51_utt1.mp4   dia76_utt3.mp4
dia103_utt0.mp4   dia23_utt2.mp4   dia51_utt2.mp4   dia76_utt4.mp4
dia103_utt1.mp4   dia23_utt3.mp4   dia51_utt3.mp4   dia76_utt5.mp4
dia103_utt2.mp4   dia23_utt4.mp4   dia51_utt4.mp4   dia76_utt6.mp4
dia103_utt3.mp4   dia23_utt5.mp4   dia51_utt5.mp4   dia76_utt7.mp4
dia103_utt4.mp4   dia23_utt6.mp4   dia51_utt6.mp4   dia76_utt8.mp4
dia104_utt0.mp4   dia23_utt7.mp4   dia52_utt0.mp4   dia76_utt9.mp4
dia104_utt1.mp4   dia23_utt8.mp4   dia52_utt1.mp4   dia77_utt0.mp4
dia104_utt10.mp4  dia23_utt9.mp4   dia52_utt2.mp4   dia77_utt1.mp4
dia104_utt11.mp4  dia24_utt0.mp4   dia53_utt0.mp4   

In [20]:
# Train
X_text_Train = train_utterances['Utterance']
X_video_dir_Train = []
X_audio_dir_Train = []
if not os.path.exists('./train_audio'):
    os.mkdir('./train_audio')
for i in range(len(train_utterances)):
    a_dir = './train_audio/dia' + str(train_utterances.iloc[i]['Dialogue_ID']) +  '_utt' + str(train_utterances.iloc[i]['Utterance_ID']) + '.wav'
    X_audio_dir_Train.append(a_dir)
    v_dir = './MELD.Raw/train_splits/dia' + str(train_utterances.iloc[i]['Dialogue_ID']) +  '_utt' + str(train_utterances.iloc[i]['Utterance_ID']) + '.mp4'
    X_video_dir_Train.append(v_dir)  

Y_E_Train = list(train_utterances['Emotion'])
Y_S_Train = list(train_utterances['Sentiment'])

# Test
X_text_Test = test_utterances['Utterance']
X_video_dir_Test = []
X_audio_dir_Test = []
if not os.path.exists('./test_audio'):
    os.mkdir('./test_audio')
for i in range(len(test_utterances)):
    a_dir = './test_audio/dia' + str(test_utterances.iloc[i]['Dialogue_ID']) +  '_utt' + str(test_utterances.iloc[i]['Utterance_ID']) + '.wav'
    X_audio_dir_Test.append(a_dir)
    v_dir = './MELD.Raw/output_repeated_splits_test/dia' + str(test_utterances.iloc[i]['Dialogue_ID']) +  '_utt' + str(test_utterances.iloc[i]['Utterance_ID']) + '.mp4'
    X_video_dir_Test.append(v_dir)  

Y_E_Test = list(test_utterances['Emotion'])
Y_S_Test = list(test_utterances['Sentiment'])

# Dev
X_text_Dev = dev_utterances['Utterance']
X_video_dir_Dev = []
X_audio_dir_Dev = []
if not os.path.exists('./dev_audio'):
    os.mkdir('./dev_audio')
for i in range(len(dev_utterances)):
    a_dir = './dev_audio/dia' + str(dev_utterances.iloc[i]['Dialogue_ID']) +  '_utt' + str(dev_utterances.iloc[i]['Utterance_ID']) + '.wav'
    X_audio_dir_Dev.append(a_dir)
    v_dir = './MELD.Raw/dev_splits_complete/dia' + str(dev_utterances.iloc[i]['Dialogue_ID']) +  '_utt' + str(dev_utterances.iloc[i]['Utterance_ID']) + '.mp4'
    X_video_dir_Dev.append(v_dir)  

Y_E_Dev = list(dev_utterances['Emotion'])
Y_S_Dev = list(dev_utterances['Sentiment'])

# Print
len(Y_E_Train) , len(Y_E_Test) , len(Y_E_Dev)

(9989, 2610, 1109)

In [21]:
! pip install -q imageio==2.4.1
! pip install -q moviepy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.0 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.


In [22]:
from moviepy.editor import *

def mp4_to_wav(direction , file_path):
    # print('Processing "%s"...' % dir)
    dest_path = file_path + direction.split('/')[-1][:-3] + 'wav'
    if not os.path.exists(dest_path):
        videoclip = VideoFileClip(direction)
        audioclip=videoclip.audio
        # print(dest_path)
        audioclip.write_audiofile(dest_path, verbose=False)
        audioclip.close()
        videoclip.close()
        #print('Processed "%s".' % dir)

In [ ]:
from tqdm.auto import tqdm

        
# train
err_dir_train = []
valid_indexes_train = []

for i in tqdm (range (len(X_video_dir_Train))):
    direction = X_video_dir_Train[i]
    try:
        mp4_to_wav(direction , './train_audio/')
        valid_indexes_train.append(i)
    except:
        err_dir_train.append(direction)

# test
err_dir_test = []
valid_indexes_test = []

for i in tqdm (range (len(X_video_dir_Test))):
    direction = X_video_dir_Test[i]
    try:
        mp4_to_wav(direction , './test_audio/')
        valid_indexes_test.append(i)
    except:
        err_dir_test.append(direction)

# dev 
err_dir_dev = []
valid_indexes_dev = []

for i in tqdm (range (len(X_video_dir_Dev))):
    direction = X_video_dir_Dev[i]
    try:
        mp4_to_wav(direction , './dev_audio/')
        valid_indexes_dev.append(i)
    except:
        err_dir_dev.append(direction)
clear_output()

In [24]:
# train test dev
print(len(X_video_dir_Train) , len(X_video_dir_Test) , len(X_video_dir_Dev))
print(len(err_dir_train) , len(err_dir_test) , len(err_dir_dev))
print(len(valid_indexes_train) , len(valid_indexes_test) , len(valid_indexes_dev))

9989 2610 1109
1 945 1
9988 1665 1108


In [25]:
from IPython.core.display import clear_output
! pip install -q vosk
!vosk-transcriber --list-models
!vosk-transcriber --list-languages
!apt -qq install ffmpeg
! sudo apt-get upgrade libncurses6 -y
clear_output()

In [26]:
! mkdir Vosk_model
os.chdir('./Vosk_model')
!wget -q https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip \*.zip
! rm vosk-model-small-en-us-0.15.zip
os.chdir('..')
clear_output()

In [27]:
import vosk
import sys
import os
import wave

import subprocess, sys, os, json

from datetime import datetime, timedelta

SAMPLE_RATE = 16000
CHUNK_SIZE = 4000

vosk_model_path = "./Vosk_model/vosk-model-small-en-us-0.15"

class Transcriber():
    def __init__(self, model_path):
        self.model = vosk.Model(model_path)

    def fmt(self, data):
        data = json.loads(data)

        start = min(r["start"] for r in data.get("result", [{ "start": 0 }]))
        end = max(r["end"] for r in data.get("result", [{ "end": 0 }]))

        return {
            "start": str(timedelta(seconds=start)), 
            "end": str(timedelta(seconds=end)), 
            "text": data["text"]
        }
    
    def transcribe(self, filename):
        rec = vosk.KaldiRecognizer(self.model, SAMPLE_RATE)
        rec.SetWords(True)

        if not os.path.exists(filename):
            raise FileNotFoundError(filename)

        transcription = []

        ffmpeg_command = [
                "ffmpeg",
                "-nostdin",
                "-loglevel",
                "quiet",
                "-i",
                filename,
                "-ar",
                str(SAMPLE_RATE),
                "-ac",
                "1",
                "-f",
                "s16le",
                "-",
            ]

        with subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE) as process:

            start_time = datetime.now() 
            while True:
                data = process.stdout.read(4000)
                if len(data) == 0:
                    break
                
                if rec.AcceptWaveform(data):
                    transcription.append(self.fmt(rec.Result()))

            transcription.append(self.fmt(rec.FinalResult()))
            end_time = datetime.now()

            time_elapsed = end_time - start_time
            
        return transcription



In [28]:
transcriber = Transcriber(vosk_model_path)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from ./Vosk_model/vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from ./Vosk_model/vosk-model-small-en-us-0.15/graph/HCLr.fst ./Vosk_model/vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo ./Vosk_model/vosk-model-small-en-us-0.15/graph/phones/word_boundary.int


In [29]:
# train
X_train_transcription = []
final_valid_indexes_train = []
for i in tqdm(valid_indexes_train):
    filename = X_audio_dir_Train[i]
    try:
        transcription = transcriber.transcribe(filename)
        X_train_transcription.append(''.join(t['text'] for t in transcription))
        final_valid_indexes_train.append(i)
    except:
        err_dir_train.append(i)
        
# test
X_test_transcription = []
final_valid_indexes_test = []


for i in tqdm(valid_indexes_test):
    filename = X_audio_dir_Test[i]
    try:
        transcription = transcriber.transcribe(filename)
        X_test_transcription.append(''.join(t['text'] for t in transcription))
        final_valid_indexes_test.append(i)
    except:
        err_dir_test.append(i)
        
# dev
X_dev_transcription = []
final_valid_indexes_dev = []

for i in tqdm(valid_indexes_dev):
    filename = X_audio_dir_Dev[i]
    try:
        transcription = transcriber.transcribe(filename)
        X_dev_transcription.append(''.join(t['text'] for t in transcription))
        final_valid_indexes_dev.append(i)
    except:
        err_dir_dev.append(i)

  0%|          | 0/9988 [00:00<?, ?it/s]

ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/con

  0%|          | 0/1665 [00:00<?, ?it/s]

ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/conda/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /opt/con

In [30]:
# train test dev
print(len(X_train_transcription) , len(X_test_transcription) , len(X_dev_transcription))
print(len(err_dir_train) , len(err_dir_test) , len(err_dir_dev))
print(len(final_valid_indexes_train) , len(final_valid_indexes_test) , len(final_valid_indexes_dev))

9988 1665 1108
1 945 1
9988 1665 1108


In [31]:
X_train_transcription[0] , final_valid_indexes_train[0] , train_utterances.iloc[final_valid_indexes[0]]

NameError: name 'final_valid_indexes' is not defined

In [32]:
# Train
train_utterances['Vosk ASR'] = ''
for i in range(len(X_train_transcription)):
    train_utterances.at[final_valid_indexes_train[i],'Vosk ASR'] = X_train_transcription[i]
    
# Test    
test_utterances['Vosk ASR'] = ''
for i in range(len(X_test_transcription)):
    test_utterances.at[final_valid_indexes_test[i],'Vosk ASR'] = X_test_transcription[i]
    
# Dev
dev_utterances['Vosk ASR'] = ''
for i in range(len(X_dev_transcription)):
    dev_utterances.at[final_valid_indexes_dev[i],'Vosk ASR'] = X_dev_transcription[i]

In [ ]:
for i in range(len(train_utterances)):
    if train_utterances.iloc[i]['Vosk ASR']=='':
        print(i , train_utterances.iloc[i]['Vosk ASR'] , train_utterances.iloc[i]['Utterance'])

In [33]:
train_utterances.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,Vosk ASR
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731",also i was the point person on my company's tr...
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442",most of edge hands full
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389",that i did that idea
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572",so let's talk a little bit about your duties
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917",my dutiesright


In [39]:
X_train_transcription_df = pd.DataFrame(train_utterances)
X_train_transcription_df.to_csv('train_transcription_df.csv')
X_train_transcription_df = pd.read_csv('train_transcription_df.csv')

X_test_transcription_df = pd.DataFrame(test_utterances)
X_test_transcription_df.to_csv('test_transcription_df.csv')
X_test_transcription_df = pd.read_csv('test_transcription_df.csv')

X_dev_transcription_df = pd.DataFrame(dev_utterances)
X_dev_transcription_df.to_csv('dev_transcription_df.csv')
X_dev_transcription_df = pd.read_csv('dev_transcription_df.csv')

In [35]:
! pwd

/kaggle/working
